# Calculating inter-model spreads in CMIP6 variables
E Boland 2021 - contact emmomp@bas.ac.uk for any issues

This notebook is designed to access the JASMIN CMIP6 holdings and calculate inter-model spreads and standard deviations in a given 2D variable. 

Requires baspy to access the CMIP6 holdings, accessible here: https://github.com/scotthosking/baspy

Options: 
- to include all runs, or only one per model, or one per modelling centre. 
- choose experiment
- choose time span

### Setting up 

In [1]:
import sys
sys.path.insert(0,'/home/users/eboland/python')
import baspy as bp
import xarray as xr
import pyresample
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

save_dir='../data_out' # where to save results
plots_dir='../plots' # where to save plots

### Defining functions to re-grid the model data

Note that these default to re-gridding everything to a 1 degree x 1 degree grid, but also take new_lat and new_lon as arguments if you want a different (regular) grid

In [2]:
#Function to get re-gridding kernel for applying at each timestep
def setup_regrid(old_lat,old_lon,new_lat=np.linspace(-89.5,89.5,180),new_lon=np.linspace(-179,180,360)):
    orig_grid = pyresample.geometry.SwathDefinition(lons=old_lon.ravel(), lats=old_lat.ravel())
    yi,xi=np.meshgrid(new_lat,new_lon)
    new_grid  = pyresample.geometry.GridDefinition(lons=xi,lats=yi)
    resample_data = pyresample.kd_tree.get_neighbour_info(orig_grid, new_grid, 200000, neighbours=1)
    return resample_data   
#Function to apply re-gridding kernel, return as xarray dataarray

def repeat_regrid(ds,resample_data,new_lon=np.linspace(-179,180,360), new_lat=np.linspace(-89.5,89.5,180),loop_dim='time'):    
    grid_shape=[new_lon.size,new_lat.size]
    stack_dims=ds.dims[1:]
    foo = pyresample.kd_tree.get_sample_from_neighbour_info('nn', grid_shape, ds.stack(z=stack_dims).transpose(...,loop_dim).values,
                                              resample_data[0], resample_data[1],resample_data[2])    
    ds2=xr.DataArray(foo,dims=['lon','lat',loop_dim],coords={'lon':(('lon'),new_lon),'lat':(('lat'),new_lat),loop_dim:(loop_dim,ds[loop_dim])})
    return ds2

In [3]:
from cmip6_preprocessing.preprocessing import rename_cmip6,promote_empty_dims,broadcast_lonlat,correct_lon,correct_coordinates,parse_lon_lat_bounds, maybe_convert_bounds_to_vertex,maybe_convert_vertex_to_bounds

def wrapper(ds):
    ds = ds.copy()
    ds = rename_cmip6(ds)
    ds = promote_empty_dims(ds)
    ds = broadcast_lonlat(ds)
   # ds = replace_x_y_nominal_lat_lon(ds)
    ds = correct_lon(ds)
    ds = correct_coordinates(ds)
    #ds = parse_lon_lat_bounds(ds)
   # ds = maybe_convert_bounds_to_vertex(ds)
   # ds = maybe_convert_vertex_to_bounds(ds)
    return ds

### 1. Choose the variable, experiment, and time span you want

In [4]:
var='hfds' #'tos','sos','hfds','tauuo','tauvo'
experiment='historical' #or rcp45 etc
time_start='1992-01-12' # In format 'YYYY-MM-DD' or None for the start/end
time_end='2015-01-12' # Need to go beyond end of
time_index=pd.date_range(time_start, time_end,freq='M')-pd.DateOffset(days=15) # Valid for monthly data

### 2. Find data
Can choose to find all runs available, or only one per model

In [5]:
#runs='all' # all runs available
runs='one' # loads only run r1i1p1 

if runs == 'all':
    print('Finding all runs for {} in {} experiment'.format(var,experiment))
    df = bp.catalogue(dataset='cmip6',Experiment=experiment,Var=var,CMOR='Omon') #All models, all centres, all ensemble members:
    dim_id='model_run' #New dimension model_run will be created to calculate spreads/std devs
elif runs=='one' :
    print('Finding run r1i1p1 for {} in {} experiment'.format(var,experiment))
    df = bp.catalogue(dataset='cmip6',Experiment=experiment,Var=var,RunID='r1i1p1f1',CMOR='Omon') # Only one ensemble per model
    dim_id = 'model'
else:
    print('Please set runs to "all" or "one"') 
models=set(df.Model)
centres=set(df.Centre)
print('Found {} sets of data from {} models from {} centres'.format(len(df),len(models),len(centres)))

Finding run r1i1p1 for hfds in historical experiment
Updating cached catalogue...
catalogue memory usage (MB): 26.848599
Found 53 sets of data from 46 models from 25 centres


### 2.a. Remove Duplicates
Remove duplicates where models are provided on two grids, favour the reporting grid 'gr'

In [6]:
model_list=[]
for model in models:
    df_subset=df.loc[df['Model']==model]    
    if len(df_subset)>1:
        # Favor reporting grid over native
        df_drop=df_subset.loc[df['Grid']=='gn'] 
        df.drop(df_drop.index,inplace=True)

models=set(df.Model)
centres=set(df.Centre)
print('Found {} sets of data from {} models from {} centres'.format(len(df),len(models),len(centres)))

Found 46 sets of data from 46 models from 25 centres


/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### 2.b. (optional) Subset models further
Choose only one model per centre, comment out this cell to keep all models

In [77]:
#model_list=[]
#for centre in centres:
#    df_subset=df.loc[df['Centre']==centre]
#    model_subset=set(df_subset.Model)
#    #choose one
#    foo=model_subset.pop()
#    model_list.append(foo)
#df=df.loc[df['Model'].isin(model_list)]
#models=set(df.Model)
#centres=set(df.Centre)
#print('Found {} sets of data from {} models from {} centres'.format(len(df),len(models),len(centres)))

### 3. Get data from each model, regridding as you go

In [ ]:
data_all=[]

for index,row in df.iterrows():
    model=row['Model']
    centre=row['Centre']
    run=row['RunID']
    
    files=bp.get_files(row)        
    print('{} {} {} Found {} files'.format(centre,model,run,len(files)))   
    data=xr.open_mfdataset(files,preprocess=wrapper)
        
    #Subset by time
    data=data.sel(time=slice(time_start,time_end))          

    
    old_lat=data['lat'].data
    old_lon=data['lon'].data
    # Pyresample requires lons between -180 and 180
    old_lon=xr.where(old_lon>180,old_lon-360,old_lon)
    old_lon=xr.where(old_lon<-180,old_lon+360,old_lon)
    
    data_in=data[var]
    # Check lon/lat and data have the same shape
    if old_lon.shape == data_in.shape[1:]:
        data_in = data_in
    else: 
        print('Transposing')
        data_in = data_in.transpose(dims=['time','x','y'])
        
    if (model=='SAM0-UNICON') and (var=='hfds'):
        data_in=-data_in
 #      
    print('Found data, proceeding to re-grid to 1 deg by 1 deg')
    resample_data=setup_regrid(old_lat,old_lon) # Generate re-gridding kernel
    data_1deg=repeat_regrid(data_in,resample_data)
    data_1deg['time']=time_index

    print('Regridding complete')

    data_1deg.attrs.update(data[var].attrs)
    
    if runs == 'all':
        data_1deg=data_1deg.assign_coords({'model_run':model+'_'+run})
    else :
        data_1deg=data_1deg.assign_coords({'model':model,'run':run})
    
    data_all.append(data_1deg)

data_all=xr.concat(data_all,dim=dim_id,combine_attrs='drop_conflicts')
data_all.attrs={'name':var,'units':data_1deg.attrs['units'],'long_name':data_1deg.attrs['long_name']}      
print('All regridding complete')

AS-RCEC TaiESM1 r1i1p1f1 Found 165 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Regridding complete
AWI AWI-CM-1-1-MR r1i1p1f1 Found 18 files


### 4. Plot regridded models to spot any issues

In [79]:
data_all.mean(dim=['time']).plot(x='lon',y='lat',col=dim_id,col_wrap=5,robust=True)
plt.suptitle('Time mean {}'.format(var),y=1.02,fontweight='bold')
plt.savefig(plots_dir+'/cmip6_time_mean_'+var+'_modelthumbnails.png')
plt.close()

### 5. Calculate and save model spread and model std-deviations

In [80]:
if var == 'hfds':
    data_all=data_all.drop_sel(model='CAS-ESM2-0') #Units off

# Take inter-model standard deviation
#dropna=False ensures only gridcells where all models have values are included
model_std = data_all.std(dim=dim_id,dropna=False)
model_std.attrs['units']=data_all.attrs['units']
model_std.attrs['name']='stddev_'+data_all.attrs['name']
model_std.attrs['long_name']='Ensemble Standard Deviation of '+data_all.attrs['long_name']
model_std.attrs['comments']='Calculated for models listed'

# Take inter-model spread
model_spread = data_all.max(dim=dim_id,dropna=False)-data_all.min(dim=dim_id,dropna=False)
model_spread.attrs['units']=data_all.attrs['units']
model_spread.attrs['name']='spread_'+data_all.attrs['name']
model_spread.attrs['long_name']='Ensemble Spread of '+data_all.attrs['long_name']
model_std.attrs['comments']='Calculated for models listed'

# Put results in one dataset, write to file
dataset_out = xr.Dataset(data_vars={'model_std':model_std,'model_spread':model_spread,'models':data_all['model']})
dataset_out.attrs['comments']='Generated on '+str(np.datetime64('today'))
dataset_out.to_netcdf((save_dir+'/cmip6_stdspread_{}_{}.nc').format(var,dim_id))

# Plot results
fig=plt.figure(figsize=[12,4])
ax=plt.subplot(1,2,1)
model_std.mean(dim='time').plot(x='lon',y='lat',robust=True,ax=ax)
ax.set_title('Time mean model std. dev. in {}'.format(var),y=1.02,fontweight='bold')     

ax=plt.subplot(1,2,2)
model_spread.mean(dim='time').plot(x='lon',y='lat',robust=True,ax=ax)
ax.set_title('Time mean model spread in {}'.format(var),y=1.02,fontweight='bold')   
plt.savefig(plots_dir+'/cmip6_stdspread_{}_{}.png'.format(var,dim_id))
plt.close()

## 6. Convert to python executable [optional]

In [1]:
!jupyter nbconvert --to script CMIP6_ranges.ipynb

[NbConvertApp] Converting notebook CMIP6_ranges.ipynb to script
[NbConvertApp] Writing 11288 bytes to CMIP6_ranges.py
